

## Project: Algorithm for animals detection usin ResNet 



In [54]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    m_files = np.array(data['filenames'])
    m_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return m_files, m_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('Images/Images/train')
valid_files, valid_targets = load_dataset('Images/Images/valid')
test_files, test_targets = load_dataset('Images/Images/test')

print('There are %d training images.' % len(train_files))
print('There are %d validation images.' % len(valid_files))
print('There are %d test images.'% len(test_files))

There are 6680 training images.
There are 835 validation images.
There are 836 test images.



## Step 2: Detect Animal

In this section, we use a pre-trained ResNet50 model to detect animals  in images. 

In [55]:
#to download the ResNet-50 model, along with weights that have been trained onImageNet
from keras.applications.resnet50 import ResNet50# to download the ResNet-50 modelwith weights when traind on ImageNet

# define ResNet50 model
ResNet50_model = ResNet50(weights='imagenet')

### Pre-process the Data



In [56]:
"""
Keras CNNs require a 4D array as input, with shape

(nb_samples,rows,columns,channels),
(nb_samples,rows,columns,channels),
 

The path_to_tensor function below takes a string-valued file path to a color image as input and returns a 
4D tensor suitable for supplying to a Keras CNN. The function first loads the image and resizes it to a 
square image that is  224×224224×224  pixels. Next, the image is converted to an array, which is then resized to a 4D tensor.
the returned tensor will always have shape


(1,224,224,3).
(1,224,224,3).
 
The paths_to_tensor function takes a numpy array of string-valued image paths as input and returns a 4D tensor with shape

(nb_samples,224,224,3).
(nb_samples,224,224,3).
 
"""

from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

### Making Predictions with ResNet-50



In [57]:
from keras.applications.resnet50 import preprocess_input, decode_predictions

def ResNet50_predict_labels(img_path):
    # returns prediction vector for image located at img_path
    img = preprocess_input(path_to_tensor(img_path))
    return np.argmax(ResNet50_model.predict(img))

### An animal preductor Detector


In [58]:
#categories corresponding to ms appear in an uninterrupted sequence and correspond to dictionary keys 151-268
### returns "True" if a m is detected in the image stored at img_path
def m_detector(img_path):
    prediction = ResNet50_predict_labels(img_path)
    return ((prediction <= 268) & (prediction >= 151)) 

### (IMPLEMENTATION) Assess the m Detector

We will use Use the code cell below to test the performance of our detector` function.  


In [59]:
import matplotlib.pyplot as plt 
import cv2
import io

In [ ]:
img_test_path="D:\Sub3_dog_breed\Sub3_dog_breed/tree.jpg"
img=cv2.imread(img_test_path,cv2.IMREAD_GRAYSCALE)
#img = io.imread(img_test_path)
#io.imshow(A)
plt.imshow(img,cmap='gray')
plt.show()
if(m_detector(img_test_path)):
            print("there is animal in the picture ")
else:
        print("there is no animal in the picture ")